In [ ]:
import os
import re

from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from time import sleep

from bot.core.exchange.http_api import ExchangeManager, BybitRestAPI
from bot.utils.files import load_tokens_from_file
from bot.utils.coinmarketcap import get_top_tokens
from bot.core.db.postgres_manager import DBManager
from bot.config.credentials import host, user, password, db_name

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

import pandas as pd
# pd.options.display.float_format = '{:.2f}'.format

import polars as pl
import polars_ols as pls

import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, coint
from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm

In [ ]:
def sort_pairs(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns([
        pl.when(pl.col("coin1") <= pl.col("coin2")).then(pl.col("coin1")).otherwise(pl.col("coin2")).alias("pair_min"),
        pl.when(pl.col("coin1") <= pl.col("coin2")).then(pl.col("coin2")).otherwise(pl.col("coin1")).alias("pair_max"),
    ]).drop('coin1', 'coin2').rename({'pair_min': 'coin1', 'pair_max': 'coin2'}).select('coin1', 'coin2', pl.exclude('coin1', 'coin2'))

In [ ]:
# get_top_tokens(limit=1000, output_file='./bot/config/top_1000_tokens.txt')

#### Скачиваем данные с Bybit

In [ ]:
async def get_data(symbol, interval, n_iters):
    df = await exc_manager.get_candles(symbol=symbol, interval=interval, n_iters=n_iters)
    df = df['bybit_linear'].sort_index()
    df['Returns'] = df['Close'].pct_change()
    df['Log_price'] = np.log(df['Close'] + 1)
    df['Log_returns'] = np.log(df['Returns'] + 1)
    return df.dropna()

In [ ]:
exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))

In [ ]:
res = await get_data(symbol='ADA_USDT', interval='1h', n_iters=1)
res.shape

In [ ]:
top_1000_tokens = load_tokens_from_file("./data/top_1000_tokens.txt")
tokens_to_del = ['USDT', 'USDC', 'LEO', 'USDe']
prices = await exc_manager.get_prices()
prices = prices['bybit_linear']

token_list = [x + '_USDT' for x in top_1000_tokens if x not in tokens_to_del]
token_list = [col for col in token_list if col in prices.keys()]
len(token_list)

In [ ]:
with open('./bot/config/tokens.txt', 'r') as file:
    current_tokens = [line.strip() for line in file.readlines()]
len(current_tokens)

In [ ]:
all_ = db_manager.get_orderbooks()
all_tokens = all_['token'].unique().to_list()

In [ ]:
# Отбираем только те токены, которые есть в БД с ордербуками
token_list = []

for token in sorted(all_tokens):
    first_date = db_manager.get_oldest_date_in_orderbook(token)
    
    if first_date < datetime(2025, 11, 12, 0, tzinfo=ZoneInfo("Europe/Moscow")):
        token_list.append(token)
len(token_list)

In [ ]:
# Настройки параметров
interval = '1d'
n_iters = 1
start_date = datetime(2025, 5, 10, tzinfo=ZoneInfo("Europe/Moscow"))
end_date = datetime(2025, 11, 19, tzinfo=ZoneInfo("Europe/Moscow"))
min_volume = 1_000_000
days_ago = 30 # За сколько последних дней смотреть волатильность
max_volatility = 52
max_price = 30

In [ ]:
main_df = pd.DataFrame()

token_dict = {'PEPE_USDT': '1000PEPE_USDT', 'SHIB_USDT': 'SHIB1000_USDT', 'FLOKI_USDT': '1000FLOKI_USDT',
             'BONK_USDT': '1000BONK_USDT', 'BTT_USDT': '1000BTT_USDT', 'XEC_USDT': '1000XEC_USDT',
             'LUNC_USDT': '1000LUNC_USDT', 'SATS_USDT': '10000SATS_USDT', 'TURBO_USDT': '1000TURBO_USDT',
             'CAT_USDT': '1000CAT_USDT'}
bad_tokens = []
good_tokens = []

for token in tqdm(token_list):
    sleep(0.5)
    df = await get_data(symbol=token, interval=interval, n_iters=n_iters)
    df = df.reset_index()

    if df.empty:
        continue
    
    first_date = df['Date'].head(1).item().floor('D')
    last_date = df['Date'].tail(1).item().floor('D')
    
    if first_date > pd.Timestamp('2025-06-10', tz='Europe/Moscow'):
        continue
    if last_date < pd.Timestamp('2025-11-10', tz='Europe/Moscow'):
        continue

    vol_mean = df[-11: -1]['Turnover'].mean()
    vol24 = df.iloc[-2]['Turnover']
    if vol24 < min_volume or vol_mean < min_volume:
        if token in current_tokens:
            print(f'{token} недостаточно торгового объёма (10d_avg: {int(vol_mean):_} / last_24h: {int(vol24):_})')
            bad_tokens.append(token)
        continue
    
    df = df[(df['Date'] > start_date) & (df['Date'] < end_date)]
    df = df.set_index('Date')
    daily_df = df[-days_ago:]
    daily_df['Max_range'] = daily_df['High'] - daily_df['Low']
    daily_df['Max_range_perc'] = (daily_df['High'] - daily_df['Low']) / daily_df['Low'] * 100

    if daily_df['Max_range_perc'].max() > max_volatility:
        if token in current_tokens:
            print(f'{token} высокая волатильность: {daily_df['Max_range_perc'].max():.1f}%')
            bad_tokens.append(token)
        continue

    if daily_df['Close'][-1] > max_price:
        continue

    main_df[f'{token[:-5]}_price'] = df['Close']
    main_df[f'{token[:-5]}_log_returns'] = df['Log_returns']
    main_df[f'{token[:-5]}_log_price'] = df['Log_price']
    good_tokens.append(token)

In [ ]:
main_df.shape

In [ ]:
len(current_tokens), len(bad_tokens), len(good_tokens)

In [ ]:
[current_tokens.remove(token) for token in bad_tokens]
len(current_tokens)

In [ ]:
len(good_tokens), len(set(good_tokens))

#### Distance approach

In [ ]:
from bot.utils.data import make_price_df_from_orderbooks_bulk, normalize
import itertools
import math

In [ ]:
exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
coin_information = exc_manager.get_instrument_data()

In [ ]:
# Создадим список из датафреймов для удобства пакетной обработки, а также получим кол-во знаков после запятой для округления
max_days = 180 # За сколько дней считать расстояние
tokens = [x[:-5] for x in sorted(good_tokens)]
dfs = []
token_std = {}
token_dist = {}

for token in tokens:
    try:
        df_token = main_df[[f'{token}_price']].rename(columns={f'{token}_price': 'price'})
        df_token['token'] = token
        df_token = pl.from_pandas(df_token, include_index=True).rename({'Date': 'time'}).tail(max_days)
    except KeyError:
        print(token)
        continue
    
    try:
        dp = coin_information['bybit_linear'][token + '_USDT']['qty_step']
    except IndexError:
        dp = 0

    token_std[token] = df_token['price'].std()
    token_dist[token] = df_token['price'].max() - df_token['price'].min()
    dfs.append(df_token)

In [ ]:
price_df = make_price_df_from_orderbooks_bulk(dfs=dfs, tokens=tokens, trunc='5m')
curr_prices = price_df.drop('time').tail(1).transpose(include_header=True, header_name='coin', column_names=['price'])
price_df.tail(1)

In [ ]:
price_df.shape, price_df.drop_nans().shape

In [ ]:
normed_df = normalize(df=price_df, method='minimax', shift_to_zero=False)

In [ ]:
# Нарисуем график с нормализованными ценами
price_cols = [c for c in normed_df.columns if c != 'time']

plt.figure(figsize=(14, 4))
for col in price_cols:
    plt.plot(normed_df['time'].to_list(), normed_df[col].to_list(), label=col)

plt.xlabel("Время")
plt.ylabel("Нормализованная цена")
plt.title("Нормализованные цены монет")
plt.grid(True)
plt.tight_layout()

In [ ]:
# Посчитаем евклидово расстояние между парами криптовалют
results = []

# Перебираем все уникальные пары колонок
for col1, col2 in itertools.combinations(price_cols, 2):
    diff_sq = (normed_df[col1] - normed_df[col2]) ** 2
    distance = math.sqrt(diff_sq.sum())
    std = np.std((normed_df[col1] - normed_df[col2]).to_numpy())
    results.append((col1, col2, distance, std))

In [ ]:
std_df = pl.DataFrame({"token": list(token_std.keys()), "std_value": list(token_std.values())})
max_dist_df = pl.DataFrame({"token": list(token_dist.keys()), "max_dist": list(token_dist.values())})

In [ ]:
dist_df = pl.DataFrame(results, schema=["coin1", "coin2", "dist", "std"], orient="row").sort('dist')
dist_df = dist_df.join(curr_prices, left_on='coin1', right_on='coin').rename({'price': 'price1'})
dist_df = dist_df.join(curr_prices, left_on='coin2', right_on='coin').rename({'price': 'price2'})
dist_df = dist_df.filter(
    (pl.col('price1') / pl.col('price2') < 5) & (pl.col('price1') / pl.col('price2') > 0.2)
)

In [ ]:
dist_df = dist_df.join(std_df, left_on="coin1", right_on="token").rename({"std_value": "std_1"})
dist_df = dist_df.join(std_df, left_on="coin2", right_on="token").rename({"std_value": "std_2"})

dist_df = dist_df.join(max_dist_df, left_on="coin1", right_on="token").rename({"max_dist": "dist_1"})
dist_df = dist_df.join(max_dist_df, left_on="coin2", right_on="token").rename({"max_dist": "dist_2"})

In [ ]:
dist_df

In [ ]:
# Добавить фичи: скорость возврата к среднему

#### Cointegration approach

In [ ]:
ret_cols = [col for col in main_df.columns if '_log_price' in col]
corr_matrix = main_df[ret_cols].corr()

In [ ]:
corr_matrix

In [ ]:
# token = 'SUSHI'
# token_df = corr_matrix[f'{token}_log_price']
# token_df.shape

In [ ]:
# token_df[abs(token_df) < 0.1]

In [ ]:
# token_df[abs(token_df) > 0.75]

In [ ]:
stacked = corr_matrix.stack().reset_index()
stacked.columns = ['stock1', 'stock2', 'correlation']
# filtered_pairs = stacked[(stacked['correlation'] > 0.6) & (stacked['stock1'] < stacked['stock2'])].reset_index(drop=True)

In [ ]:
results = []
processed = []
n_rows = 180 # Сколько последних записей учитывать
total = len(list(itertools.combinations(price_cols, 2)))

# for _, row in tqdm(stacked.iterrows(), total=stacked.shape[0]):
for col1, col2 in tqdm(itertools.combinations(price_cols, 2), total=total):
    stock1 = col1 + '_log_price'
    stock2 = col2 + '_log_price'
    
        
    corr_value = pl.DataFrame(stacked).filter(
            (pl.col('stock1') == stock1) & (pl.col('stock2') == stock2)
        ).select('correlation').item()
    

    data = main_df[[stock1, stock2]].dropna()
    data = data[-n_rows:]

    if data.empty:
        print(f"Пара {stock1}-{stock2} пропущена, т.к. нет данных после очистки")
        continue

    adf_stat_1, p_value_1, _ = coint(data[stock1], data[stock2], trend='c')
    adf_stat_2, p_value_2, _ = coint(data[stock2], data[stock1], trend='c')

    beta_1 = np.cov(data[stock1].values, data[stock2].values)[0, 1] / np.var(data[stock1].values)
    beta_2 = np.cov(data[stock2].values, data[stock1].values)[0, 1] / np.var(data[stock2].values)

    results.append({
        'coin1': col1,
        'coin2': col2,
        'corr': corr_value,
        'adf_1': adf_stat_1,
        'pv_1': p_value_1,
        'adf_2': adf_stat_2,
        'pv_2': p_value_2,
        'beta_1': beta_1,
        'beta_2': beta_2
    })

    processed.append((stock1, stock2))

result_df = pd.DataFrame(results)

In [ ]:
coint_df = pl.DataFrame(result_df)
coint_df

In [ ]:
# dist_df = sort_pairs(dist_df)
# coint_df = sort_pairs(coint_df)

In [ ]:
pair_stats_df = dist_df.drop('price1', 'price2').join(coint_df, on=['coin1', 'coin2'], how='left')
pair_stats_df = pair_stats_df.filter(
    (pl.col('pv_1') < 0.05) & (pl.col('pv_2') < 0.05) & 
    (pl.col('dist') > 0.8) & (pl.col('dist') < 2.8) & (abs(pl.col('beta_1') - pl.col('beta_2')) < 1.6)
).rename({'dist': 'spr_dist', 'std': 'spr_std'}).drop('adf_1', 'adf_2')

In [ ]:
pair_stats_df

In [ ]:
# Проверка, есть ли такая пара в нашей выборке
token_1 = 'ARKM'
token_2 = 'OP'
pair_stats_df.filter(
    ((pl.col('coin1') == token_1) & (pl.col('coin2') == token_2)) | 
    ((pl.col('coin2') == token_1) & (pl.col('coin1') == token_2))
)

In [ ]:
pair_stats_df.write_parquet('./data/pair_selection/curr_pairs.parquet')

In [ ]:
# pair_stats_df = pl.read_parquet('./data/pair_selection/curr_pairs.parquet')

In [ ]:
pair_stats_df.filter(
    ((pl.col('coin1') == 'CELO') & (pl.col('coin2') == 'POL')) | 
    ((pl.col('coin1') == 'SEI') & (pl.col('coin2') == 'DYDX'))   |
    ((pl.col('coin1') == 'IOTA') & (pl.col('coin2') == 'MANA'))   |
    ((pl.col('coin1') == 'FLOW') & (pl.col('coin2') == 'MANTA'))    
)

In [ ]:
pairs = db_manager.get_table('pairs', df_type='polars')

In [ ]:
token_list = []

for row in pair_stats_df.iter_rows(named=True):
    t1 = row["coin1"]
    t2 = row["coin2"]

    if (t1, t2) not in token_list and (t2, t1) not in token_list:
        token_list.append((t1, t2))
len(token_list)

In [ ]:
for row in pairs.iter_rows(named=True):
    t1 = row['token_1'][:-5]
    t2 = row['token_2'][:-5]
    
    if (t1, t2) not in token_list and (t2, t1) not in token_list:
        token_list.append((t1, t2))
        print((t1, t2))

In [ ]:
token_list

In [ ]:
with open('./data/token_pairs.txt', 'w') as file:
    for pair in token_list:
        file.write(f"{pair[0]} {pair[1]}\n")